# Twist.

> Create and tune models with Twist layers.

In [ ]:
#hide
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import OrderedDict

from model_constructor.net import Net
from model_constructor.layers import ConvLayer, noop, act_fn, SimpleSelfAttention

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [ ]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

## ConvTwist

In [ ]:
from model_constructor.twist import ConvTwist

In [ ]:
# hide_input
print_doc(ConvTwist)

<h4 id="ConvTwist" class="doc_header"><code>class</code> <code>ConvTwist</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/twist.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvTwist</code>(**`ni`**, **`nf`**, **`ks`**=*`3`*, **`stride`**=*`1`*, **`padding`**=*`1`*, **`bias`**=*`False`*, **`groups`**=*`1`*, **`iters`**=*`1`*, **`init_max`**=*`0.7`*, **\*\*`kwargs`**) :: `Module`

Replacement for Conv2d (kernelsize 3x3)

In [ ]:
ConvTwist(64,64)

ConvTwist(
  twist: False, permute: True, same: True, groups: 8
  (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
)

In [ ]:
ConvTwist.twist, ConvTwist.permute

(False, True)

In [ ]:
ConvTwist.use_groups, ConvTwist.groups_ch

(True, 8)

In [ ]:
ConvTwist(64,64)

ConvTwist(
  twist: False, permute: True, same: True, groups: 8
  (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
)

In [ ]:
ConvTwist.twist = True
ConvTwist.permute = False
ConvTwist(64,64)

ConvTwist(
  twist: True, permute: False, same: True, groups: 8
  (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
)

## ConvLayerTwist

In [ ]:
class ConvLayerTwist(ConvLayer):  # replace Conv2d by Twist
    Conv2d = ConvTwist

In [ ]:
ConvLayerTwist(64,64, stride=1)

ConvLayerTwist(
  (conv): ConvTwist(
    twist: True, permute: False, same: True, groups: 8
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ConvLayer.Conv2d

torch.nn.modules.conv.Conv2d

In [ ]:
ConvLayerTwist.Conv2d

model_constructor.twist.ConvTwist

In [ ]:
conv_layer = ConvLayerTwist(32, 64)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: True, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ConvTwist.twist = False
conv_layer = ConvLayerTwist(32, 64)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, act=False)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, bn_layer=False)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, bn_1st=True)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, bn_1st=True, act_fn=nn.LeakyReLU())
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, ks=1)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, ks=1, stride=2)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayerTwist(32, 64, stride=2)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 4
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=4, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ConvTwist.groups_ch = 4
conv_layer = ConvLayerTwist(32, 64, stride=2)
conv_layer

ConvLayerTwist(
  (conv): ConvTwist(
    twist: False, permute: False, same: False, groups: 8
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=8, bias=False)
  )
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

## NewResBlockTwist

In [ ]:
from model_constructor.twist import NewResBlockTwist

In [ ]:
# hide_input
print_doc(NewResBlockTwist)

<h4 id="NewResBlockTwist" class="doc_header"><code>class</code> <code>NewResBlockTwist</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/twist.py#L109" class="source_link" style="float:right">[source]</a></h4>

> <code>NewResBlockTwist</code>(**`expansion`**, **`ni`**, **`nh`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **`zero_bn`**=*`True`*, **\*\*`kwargs`**) :: `Module`

Resnet block with ConvTwist.
Reduce by pool instead of stride 2.
Now YaResBlock.

In [ ]:
#collapse_output
bl = NewResBlockTwist(4,64,64,sa=True)
bl

NewResBlockTwist(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: True, groups: 16
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [ ]:
#collapse_output
bl = NewResBlockTwist(4,64,64,stride=2)
bl

NewResBlockTwist(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: True, groups: 16
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): ReLU(inplace=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 16, 16]), f"size"

torch.Size([16, 256, 16, 16])


In [ ]:
#collapse_output
bl = NewResBlockTwist(4,64,128,stride=2)
bl

NewResBlockTwist(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: True, groups: 32
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      )
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), 

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [ ]:
#hide
bl = NewResBlockTwist(1,64,64,sa=True)
bl

NewResBlockTwist(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
#collapse_output
bl = NewResBlockTwist(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False)
bl

NewResBlockTwist(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: True, groups: 32
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      )
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


## ResBlockTwist

In [ ]:
from model_constructor.twist import ResBlockTwist

In [ ]:
#hide_input
print_doc(ResBlockTwist)

<h4 id="ResBlockTwist" class="doc_header"><code>class</code> <code>ResBlockTwist</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/twist.py#L138" class="source_link" style="float:right">[source]</a></h4>

> <code>ResBlockTwist</code>(**`expansion`**, **`ni`**, **`nh`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **\*\*`kwargs`**) :: `Module`

Resnet block with ConvTwist

In [ ]:
#collapse_output
bl = ResBlockTwist(4,64,64,sa=True)
bl

ResBlockTwist(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: True, groups: 16
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [ ]:
#collapse_output
bl = ResBlockTwist(4,64,64,stride=2)
bl

ResBlockTwist(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: False, groups: 16
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 16, 16]), f"size"

torch.Size([16, 256, 16, 16])


In [ ]:
#collapse_output
bl = ResBlockTwist(4,64,128,stride=2)
bl

ResBlockTwist(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1_twist): ConvLayerTwist(
      (conv): ConvTwist(
        twist: False, permute: False, same: False, groups: 32
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=32, bias=False)
      )
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


## Model

In [ ]:
model  = Net(expansion=4, layers=[3,4,6,3])

In [ ]:
model.block = NewResBlockTwist

In [ ]:
#collapse_output
model.body

Sequential(
  (l_0): Sequential(
    (bl_0): NewResBlockTwist(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1_twist): ConvLayerTwist(
          (conv): ConvTwist(
            twist: False, permute: False, same: True, groups: 16
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          )
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (idconv): ConvLayer(
    

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


/home/jzz/anaconda3/envs/mc_dev/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [ ]:
model.block = ResBlockTwist

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [ ]:
m = model()

In [ ]:
#collapse_output
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): ResBlockTwist(
        (convs): Sequential(
      

In [ ]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = m(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


In [ ]:
#collapse_output
m.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
#collapse_output
m.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)

In [ ]:
#collapse_output
m.body.l_0

Sequential(
  (bl_0): ResBlockTwist(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1_twist): ConvLayerTwist(
        (conv): ConvTwist(
          twist: False, permute: False, same: True, groups: 16
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        )
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=

In [ ]:
#collapse_output
m.body.l_1

Sequential(
  (bl_0): ResBlockTwist(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1_twist): ConvLayerTwist(
        (conv): ConvTwist(
          twist: False, permute: False, same: False, groups: 32
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=32, bias=False)
        )
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLayer(
    

In [ ]:
#collapse_output
m.body.l_2

Sequential(
  (bl_0): ResBlockTwist(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1_twist): ConvLayerTwist(
        (conv): ConvTwist(
          twist: False, permute: False, same: False, groups: 64
          (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
        )
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLayer(
  

In [ ]:
#collapse_output
m.body.l_3

Sequential(
  (bl_0): ResBlockTwist(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1_twist): ConvLayerTwist(
        (conv): ConvTwist(
          twist: False, permute: False, same: False, groups: 128
          (conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
        )
        (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLayer(

model_constructor
by ayasyrev